![](Bellman.png)

### Ecuación de actualización en Q-learning:

### ¿Qué significa?

Esta ecuación **actualiza el valor de hacer una acción `a` en un estado `s`** con base en lo que el agente experimenta.
Cada vez que el agente se mueve y observa una recompensa, mejora su "tabla de decisiones" llamada **Q-table**.

### ¿Qué representa cada término?

| Símbolo              | Significado                                                                                                                   |
| -------------------- | ----------------------------------------------------------------------------------------------------------------------------- |
| `Q(s, a)`            | Valor actual estimado de hacer la acción `a` en el estado `s`.                                                                |
| `α` (alfa)           | Tasa de aprendizaje: cuánto confías en lo nuevo que acabas de aprender (valor entre 0 y 1).                                   |
| `R(s, a)`            | Recompensa inmediata que obtuviste por hacer `a` en `s`.                                                                      |
| `γ` (gamma)          | Factor de descuento: cuánto te importa el futuro. Si es 0, solo importa el presente. Si es cercano a 1, te importa el futuro. |
| `max_{a'} Q(s', a')` | Mejor valor posible desde el nuevo estado `s'`, mirando todas las acciones posibles `a'`.                                     |

---

### En palabras simples:

> “Actualiza el valor de esta acción con lo que ya sabías, **más un pedacito de lo que aprendiste nuevo**: la recompensa obtenida y lo que esperas ganar en el futuro.”

- Permite a un agente imaginario aprender a tomar decisiones óptomas y a alcanzar un objetivo en ese entorno o ambiente.
- Aprende con los valores que sean más convenientes en cada paso. A estos pasos los implementamos en un par de datos que definen la acción que tiene que tomar y el estado en el que queda.

- Agente que debe identificar que pasos realizar
- Calidad de los pasos: Q

- Valores Q ayudan al agente a tomar una decisión en cada paso.

In [1]:
import numpy as np
import random

In [2]:
# Definición del entorno
# ----------------------

# Se establece una cuadrícula de 5x5 donde se desarrollará el proceso de aprendizaje.
dimensiones = (5, 5)

# El agente comienza en la celda (0, 0).
estado_inicial = (0, 0)

# El objetivo es llegar a la celda (4, 4).
estado_objetivo = (4, 4)

# Algunas celdas están bloqueadas por obstáculos y no son transitables.
obstaculos = [(1, 1), (1, 3), (2, 3), (3, 0)]

# Acciones disponibles: arriba, abajo, izquierda, derecha.
# Cada acción es una tupla (Δfila, Δcolumna).
acciones = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [3]:
# Se calcula el número total de estados (celdas).
num_estados = dimensiones[0] * dimensiones[1]
num_estados

25

In [4]:
# Y el número total de acciones posibles por estado.
num_acciones = len(acciones)
num_acciones

4

### Inicialización de la tabla Q
- Se crea una matriz de dimensiones (número de estados) x (número de acciones),
- donde cada celda Q[s, a] representa el valor esperado de tomar la acción 'a' en el estado 's'.


- Ejemplo:
  - Q[12, 2] te diría: ¿Cuál es el valor esperado de estar en la celda que representa el estado 12 y moverse, por ejemplo, a la izquierda (acción 2)?

In [5]:
# Crea una matriz Q de ceros, donde cada fila representa un estado y cada columna una acción.
# Esta matriz guardará los valores Q, es decir, la utilidad esperada de tomar una acción en un estado.
Q = np.zeros((num_estados, num_acciones))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

- Toma una tupla estado = (fila, columna) y devuelve un entero que representa el número del estado en la Q-table.

In [7]:
# Función de mapeo: estado a índice de la Q-table
# ------------------------------------------------
# Convierte un estado (fila, columna) en un índice entero correspondiente
# a la fila de la tabla Q. Esto permite representar la cuadrícula 2D como una lista 1D.

def estado_a_indice(estado):
    return estado[0] * dimensiones[1] + estado[1]

In [8]:
ejemplo = estado_a_indice((0, 0))
print(ejemplo)

ejemplo = estado_a_indice((1, 0))
print(ejemplo)

ejemplo = estado_a_indice((0, 1))
print(ejemplo)

ejemplo = estado_a_indice((3, 3))
print(ejemplo)

0
5
1
18


- Alpha: 
  - Factor de la taza de aprendizaje. 
  - Cuánto se actualiza el valor Q en cada paso. 
  - Valor bajo es más lento pero más seguro
- Gamma:
  - Factor de descuento. 
  - Determinar la importancia de las recompensas que va a obtener en el futuro.
  - Cercano a 1 hace que las recompensas futuras sean casi tan importantes como las inmediatas, haciendo que el agente considera consecuencias a largo plazo de sus acciones.
  - Valos más bajo, al agente va a valorar más las consecuencias inmediatas.
- Epsilon:
  - Sirve para que el agente no repita siempre las mismas decisiones.
  - Se define la probabilidad de que el agente tome una acción aleatoria en lugar de que el agente tome una acción conocida en la tabla Q.
  - Permite que el agente explore el entorno.
- Episodios:
  - Define el número total de veces que se va a repetir el proceso de entrenamiento.
  - Empieza con el agente en el estado inicial y termina cuando alcance el objetivo.

In [9]:
alpha = 0.1
gamma = 0.99
epsilon = 0.2
episodios = 100

- Esta función aplica la estrategia ε-greedy, una técnica muy común en Q-learning para balancear:
  - Exploración (probar nuevas acciones para descubrir recompensas) 
  - y Explotación (usar el conocimiento actual para tomar la mejor decisión).

In [10]:
# Función para elegir una acción con estrategia ε-greedy
# ------------------------------------------------------
# Esta función implementa el balance entre exploración y explotación.
# Con probabilidad 'epsilon', se elige una acción aleatoria (exploración).
# Con probabilidad '1 - epsilon', se elige la mejor acción conocida
# (la de mayor valor Q en la tabla para ese estado).
# La función devuelve el índice de la acción elegida (0, 1, 2 o 3), según si el agente explora o explota.

def elegir_accion(estado):
    if random.uniform(0, 1) < epsilon:
        return random.choice(range(0, num_acciones)) # Si explora (elige aleatoriamente)
    else:
        return np.argmax(Q[estado_a_indice(estado)]) # Si explota (elige la mejor conocida). Retorna el índice de la acción con mayor valor Q

In [11]:
def aplicar_accion(estado, accion_idx):
    
    accion = acciones[accion_idx] # Se convierte el índice en una tupla que representa el movimiento.

    # Calcula la nueva posición (fila, columna) del agente después de aplicar una acción.
    # np.add(estado, accion): Suma elemento a elemento la posición actual y el movimiento. np.add((2, 2), (1, 0)) → array([3, 2])
    # % dimensiones hace que si te pasas del límite de filas o columnas, vuelvas al otro lado del tablero. np.add((2, 2), (1, 0)) % (5, 5) → array([3, 2])  (sigue dentro)
    # Convierte el resultado final de np.array a una tupla de Python
    nuevo_estado = tuple(np.add(estado, accion) % dimensiones)

    # Si el nuevo estado es un obstáculo o si el movimiento resultó en quedarse en el mismo lugar, entonces:
    # No se mueve (estado se mantiene),
    # Recibe una penalización fuerte: -100,
    # El episodio no termina aún (False).
    if nuevo_estado in obstaculos or nuevo_estado == estado:
        return estado, -100, False

    # Si el agente alcanza el estado objetivo:
    # Se actualiza el estado correctamente,
    # Recibe una gran recompensa de +100,
    # Y se termina el episodio (True).
    if nuevo_estado == estado_objetivo:
        return nuevo_estado, 100, True
    
    # Caso general
    # Movimiento válido, pero no llegó al objetivo.
    # Penalización leve -1 para motivar que no se quede dando vueltas innecesarias.
    # El episodio continúa (False).
    return nuevo_estado, -1, False


- Esta parte es el motor del algoritmo, donde el agente explora, se equivoca, acierta y va mejorando la tabla Q.

In [ ]:
for episodio in range(episodios): # Repetir por cada episodio (ciclo de entrenamiento)

    estado = estado_inicial
    terminado = False

    while not terminado:
        
        # Convierte la posición (fila, columna) al índice numérico que corresponde a la fila en Q.
        idx_estado = estado_a_indice(estado) 

        # Decide si explorar o explotar (ε-greedy), y elige una acción (índice 0 a 3).
        accion_idx = elegir_accion(estado)

        # Aplicar acción y obtener resultados
        nuevo_estado, recompensa, terminado = aplicar_accion(estado, accion_idx)

        # Calcular índice del nuevo estado. Necesario para actualizar Q con base en lo que viene.
        idx_nuevo_estado = estado_a_indice(nuevo_estado)

        # Actualizar la tabla Q. Se está aprendiendo cuánto vale tomar esa acción desde ese estado.
        Q[idx_estado, accion_idx] = Q[idx_estado, accion_idx] + alpha * (recompensa + gamma * np.max(Q[idx_nuevo_estado]) - Q[idx_estado, accion_idx])
        
        #  Moverse al nuevo estado
        estado = nuevo_estado

In [13]:
def mostrar_q_table():
    print("Tabla Q (valores por estado y acción):")
    for fila in range(dimensiones[0]):
        for col in range(dimensiones[1]):
            estado = (fila, col)
            idx = estado_a_indice(estado)
            print(f"Estado {estado} ({idx}): {Q[idx]}")

mostrar_q_table()

Tabla Q (valores por estado y acción):
Estado (0, 0) (0): [97.93321383 -0.15760053 18.60181009  9.70406378]
Estado (0, 1) (1): [  0.66303064 -18.30815603  42.73183634   0.        ]
Estado (0, 2) (2): [-0.1 -0.1  0.   0. ]
Estado (0, 3) (3): [0. 0. 0. 0.]
Estado (0, 4) (4): [71.75704635  1.55984524  0.          0.        ]
Estado (1, 0) (5): [ -0.1         -0.19         3.75857798 -10.        ]
Estado (1, 1) (6): [0. 0. 0. 0.]
Estado (1, 2) (7): [-0.1 -0.1  0.   0. ]
Estado (1, 3) (8): [0. 0. 0. 0.]
Estado (1, 4) (9): [24.5296263   0.         -7.95784019  0.        ]
Estado (2, 0) (10): [ -0.16072032 -19.          -0.1         -0.1099    ]
Estado (2, 1) (11): [-10.      -0.1     -0.1999  -0.1   ]
Estado (2, 2) (12): [-0.1 -0.1 -0.1  0. ]
Estado (2, 3) (13): [0. 0. 0. 0.]
Estado (2, 4) (14): [-0.1  0.   0.   0. ]
Estado (3, 0) (15): [0. 0. 0. 0.]
Estado (3, 1) (16): [ -0.19  -0.1  -10.    -0.1 ]
Estado (3, 2) (17): [-0.1    -0.1    -0.1099  0.    ]
Estado (3, 3) (18): [0. 0. 0. 0.]
Estad

In [14]:
def mostrar_politica():
    simbolos_accion = {
        0: '↑',   # (-1, 0)
        1: '↓',   # (1, 0)
        2: '←',   # (0, -1)
        3: '→'    # (0, 1)
    }

    print("\nPolítica aprendida (mejor acción por estado):\n")
    for fila in range(dimensiones[0]):
        linea = ""
        for col in range(dimensiones[1]):
            estado = (fila, col)
            if estado in obstaculos:
                linea += " ⛔ "  # obstáculo
            elif estado == estado_objetivo:
                linea += " 🎯 "  # objetivo
            else:
                idx = estado_a_indice(estado)
                mejor_accion = np.argmax(Q[idx])
                linea += f" {simbolos_accion[mejor_accion]}  "
        print(linea)

mostrar_politica()


Política aprendida (mejor acción por estado):

 ↑   ←   ←   ↑   ↑  
 ←   ⛔  ←   ⛔  ↑  
 ←   ↓   →   ⛔  ↓  
 ⛔  ↓   →   ↑   ↑  
 ←   ←   →   ↑   🎯 


In [15]:
politica = np.zeros(dimensiones, dtype=int)

for i in range(dimensiones[0]):
    for j in range(dimensiones[1]):
        estado = (i, j)
        idx_estado = estado_a_indice(estado)
        mejor_accion = np.argmax(Q[idx_estado])
        
        politica[i, j] = mejor_accion

print("Politica aprendida (0:arriba, 1:abajo, 2:izquierda, 3:derecha):")
print(politica)

Politica aprendida (0:arriba, 1:abajo, 2:izquierda, 3:derecha):
[[0 2 2 0 0]
 [2 0 2 0 0]
 [2 1 3 0 1]
 [0 1 3 0 0]
 [2 2 3 0 0]]


### Ejercicios

In [16]:
import numpy as np
import random

# Parámetros globales para la función elegir_accion
epsilon = 0.1
num_acciones = 4
Q = np.zeros((25, num_acciones))  # Ejemplo de matriz Q
dimensiones = (5, 5)

def estado_a_indice(estado):
    # Convierte el estado en un índice para la matriz Q
    return estado[0] * dimensiones[1] + estado[1]

def elegir_accion(estado):
    # Implementa la estrategia epsilon-greedy
    if random.uniform(0,1) < epsilon:
        return random.choice(range(0, num_acciones))
    else:
        return np.argmax(Q[estado_a_indice(estado)])

    